In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install swig

ERROR: Could not find a version that satisfies the requirement swig (from versions: none)
ERROR: No matching distribution found for swig


In [ ]:
!pip install stable-baselines3[extra]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


INFO: pip is looking at multiple versions of stable-baselines3[extra] to determine which version is compatible with other requirements. This could take a while.


In [ ]:
!pip install gymnasium["all"]

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
class PlotCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.rewards = []
        self.losses = []
        self.returns = []

    def _on_step(self) -> bool:
        if hasattr(self.model, 'logger'):
            loss = self.model.logger.name_to_value.get('train/loss', np.nan)
            if not np.isnan(loss):
                self.losses.append(loss)
        return True

    def _on_rollout_end(self) -> None:
        # Log rewards and calculate returns at the end of each episode
        if hasattr(self.model, 'ep_info_buffer') and self.model.ep_info_buffer:
            for ep_info in self.model.ep_info_buffer:
                self.rewards.append(ep_info['r'])

        self.model.ep_info_buffer.clear()

In [ ]:
env = gym.make('Pusher-v5')

In [ ]:
gamma = 0.99
learning_rate = 0.0001

In [ ]:
# implement td3 on pusher
from stable_baselines3 import TD3

In [ ]:
model = TD3('MlpPolicy', env, verbose=1, gamma=gamma, learning_rate=learning_rate)

In [ ]:
callback = PlotCallback()
training_history = model.learn(total_timesteps=200000, callback=callback, log_interval=1000)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 2)
plt.plot(callback.rewards, label="Returns")
plt.xlabel("Episode")
plt.ylabel("Total Reward")

plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
model.save("td3_pusher1")